In [42]:
from torch.utils.data import DataLoader
import pickle
# from datasets_v02 import fNIRSChannelSpaceSegmentLoad, fNIRSPreloadDataset
import torch
import os
import xarray as xr
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
# from model import CNN2DImage, CNN2DChannelV2, CNN2D_BaselineV2
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
# from utils import create_train_test_files, create_train_test_segments, create_train_test_segments_grad
warnings.filterwarnings("ignore")


print("hello hydra")
print(f"job ID: {os.getenv('SLURM_JOB_ID')}")
print(f"array job ID: {os.getenv('SLURM_ARRAY_JOB_ID')}")
print(f"array task ID: {os.getenv('SLURM_ARRAY_TASK_ID')}")
print(f"CUDA available: {torch.cuda.is_available()}")



hello hydra
job ID: 3795443
array job ID: None
array task ID: None
CUDA available: True


In [43]:
class CNN2DImage(nn.Module):
    def __init__(self):
        super().__init__()

        self.feature_extractor = nn.Sequential(
            # nn.Conv2d(104, 64, kernel_size=(1, 3)), (shakiba's)
            # nn.Conv2d(110, 64, kernel_size=(1, 3)), # parcel space
            nn.Conv2d(68, 64, kernel_size=(1, 3)), # channel space (FreshMotor)
            # nn.Conv2d(100, 64, kernel_size=(1, 3)), # channel space (BallSqueezingHD)
            nn.ReLU(),
            nn.Dropout(0.6),
            nn.InstanceNorm2d(64),
            nn.MaxPool2d(kernel_size=(1, 3)),
            nn.Conv2d(64, 32, kernel_size=(1, 3)),
            nn.ReLU(),
            nn.Dropout(0.6),
            nn.InstanceNorm2d(32),
            nn.MaxPool2d(kernel_size=(1, 3)),
            nn.Conv2d(32, 16, kernel_size=(1, 3)),
            nn.ReLU(),
            nn.Dropout(0.6),
            nn.InstanceNorm2d(16),
            nn.MaxPool2d(kernel_size=(1, 3)),
        )

        # Don't define Linear layers yet
        self.classifier = None

    def forward(self, x):
        # print(x.shape)
        x = self.feature_extractor(x)

        if self.classifier is None:
            # First pass — define classifier based on actual input
            flattened_size = x.view(x.size(0), -1).size(1)
            self.classifier = nn.Sequential(
                nn.Flatten(start_dim=1),
                nn.Dropout(0.5),
                nn.Linear(flattened_size, 64),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(64, 2)
            )
            # Move to same device as input
            self.classifier.to(x.device)

        x = self.classifier(x)
        return x

In [44]:
from torch.utils.data import Dataset
class fNIRSPreloadDataset(Dataset):
    def __init__(self, data_csv_path, mode="train", chromo="HbO"):
        self.data_csv = pd.read_csv(data_csv_path)
        self.mode = mode
        self.chromo = chromo

        # === Pre-load all trials into RAM ===
        self.all_trials = []
        self.all_labels = []

        print(f"Preloading {len(self.data_csv)} trials into memory...")

        for i, row in self.data_csv.iterrows():
            if chromo == "both":
                record = xr.open_dataarray(row["snirf_file"])
                trial_tensor = torch.tensor(record.values, dtype=torch.float32)
            else:
                try:
                    record = xr.open_dataarray(row["snirf_file"]).sel(chromo=chromo)
                    current_len = record.shape[1]
                    target_len = 87

                    # only pad if shorter than target
                    if current_len < target_len:
                        print("Padding trial from length", current_len, "to", target_len)
                        pad_width = [(0, 0), (0, target_len - current_len)]
                        record = xr.DataArray(
                            np.pad(record.values, pad_width, mode='constant', constant_values=0),
                            dims=record.dims,
                            coords={
                                record.dims[0]: record.coords[record.dims[0]].values,
                                record.dims[1]: np.arange(target_len)
                            }
                        )
                    trial_tensor = torch.tensor(record.values, dtype=torch.float32).unsqueeze(1)

                except Exception as e:
                    print(f"Error loading {row['snirf_file']}: {e}")
                    continue
            label_tensor = torch.tensor(int(row["trial_type"]), dtype=torch.long)

            self.all_trials.append(trial_tensor)
            self.all_labels.append(label_tensor)

        print(f"Loaded {len(self.all_trials)} trials into memory.")

    def __len__(self):
        return len(self.all_trials)

    def __getitem__(self, idx):
        return self.all_trials[idx], self.all_labels[idx]

In [45]:
import glob
def create_train_test_segments(bids_path, preprocessed_path, test_subjects_list=None, test_subject_percentage=0.2, exclude_subjects=None):
    """
    Create train and test files for the dataset.
    """
    # Load the participants.tsv file
    if bids_path is not None:
        participants_df = pd.read_csv(os.path.join(bids_path, "participants.tsv"), sep="\t")
    else:
        participants = glob.glob(preprocessed_path + "/sub-*")
        participants = [os.path.basename(p) for p in participants]
        participants_df = pd.DataFrame({"participant_id": participants})

    if exclude_subjects is not None:
        participants_df = participants_df[~participants_df["participant_id"].isin(exclude_subjects)]
        print(f"Excluding subjects: {exclude_subjects}")

    if test_subjects_list is not None:
        for test_subject in test_subjects_list:
            if test_subject not in participants_df["participant_id"].values:
                raise ValueError(f"Test subject {test_subject} not found in participants.tsv")
    else:
        # Randomly select test subjects
        num_test_subjects = int(len(participants_df) * test_subject_percentage)
        test_subjects = participants_df.sample(n=num_test_subjects, random_state=42)["participant_id"].values
        test_subjects_list = list(test_subjects)

    train_subjects_list = [sub for sub in participants_df["participant_id"].values if sub not in test_subjects_list]
    print(f"Number of train subjects: {len(train_subjects_list)}")


    test_df = pd.DataFrame()
    train_df = pd.DataFrame()

    labels = {"Left": 1, "Right": 0, "left": 1, "right": 0}

    train_files =  []
    for train_subject in train_subjects_list:
        # train_files += glob.glob(os.path.join(preprocessed_path, train_subject, "**", "*.nc"))
        train_files += glob.glob(os.path.join(preprocessed_path, train_subject, "**", "*.nc"), recursive=True)
    train_labels = []
    for f in train_files:
        if os.path.basename(f).endswith("_test.nc"):
            train_labels.append(labels[os.path.basename(f).split("_")[-3]])
        else:
            train_labels.append(labels[os.path.basename(f).split("_")[-2]])
    train_df = pd.DataFrame({
        "snirf_file": train_files,
        "trial_type": train_labels})
    
    test_files =  []
    for test_subject in test_subjects_list:
        # test_files += glob.glob(os.path.join(preprocessed_path, test_subject, "**", "*_test.nc"))
        test_files += glob.glob(os.path.join(preprocessed_path, test_subject, "**", "*_test.nc"), recursive=True)
    test_labels = []
    for f in test_files:
        if os.path.basename(f).endswith("_test.nc"):
            test_labels.append(labels[os.path.basename(f).split("_")[-3]])
        else:
            test_labels.append(labels[os.path.basename(f).split("_")[-2]])
    test_df = pd.DataFrame({
        "snirf_file": test_files,
        "trial_type": test_labels})

    # Save the test and train DataFrames to CSV files
    test_df.to_csv(os.path.join(preprocessed_path, "test_segments.csv"), index=False)
    train_df.to_csv(os.path.join(preprocessed_path, "train_segments.csv"), index=False)
    return os.path.join(preprocessed_path, "train_segments.csv"), os.path.join(preprocessed_path, "test_segments.csv")

In [47]:
# Training function
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for data, labels in train_loader:
        data, labels = data.to(device), labels.to(device)

        # Forward pass
        outputs = model(data)   
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    f1_avg = []
    acc_avg = []

    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            f1_avg.append(f1_score(labels.cpu().numpy(), predicted.cpu().numpy(), average='micro'))
            acc_avg.append((predicted == labels).sum().item() / labels.size(0))

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    f1 = f1_score(all_labels, all_preds, average='micro')  # or 'macro' if you prefer
    return total_loss / len(test_loader), accuracy, f1, np.mean(f1_avg), np.mean(acc_avg)

# Main function
if __name__ == "__main__":
    # Hyperparameters
    num_epochs = 400
    learning_rate = 1e-4
    batch_size = 16
    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load datasets
    base_dir = "/home"
    # dataset_path = os.path.join(base_dir, "data/BallSqueezingHD_modified")
    # preprocessed_path = os.path.join(base_dir, "data/yuanyuan_v2_processed_partial/")
    # preprocessed_path = os.path.join("datasets/processed/BallSqueezingHD_modified/")
    preprocessed_path = os.path.join("datasets/processed/FreshMotor/")
    
    # subject_ids = ['sub-170', 'sub-173', 'sub-176', 'sub-179',
    #             'sub-182', 'sub-577', 'sub-581', 'sub-586',  
    #             'sub-613', 'sub-619', 'sub-633', 'sub-177',  
    #             'sub-181', 'sub-183', 'sub-185', 'sub-568', 
    #             'sub-580', 'sub-583', 'sub-587', 'sub-592',  
    #             'sub-618', 'sub-621', 'sub-638', 'sub-640']
    
    # subject_ids = ['sub-170', 'sub-173', 'sub-171', 'sub-174', 'sub-176', 'sub-179',
    #             'sub-182', 'sub-177', 'sub-181', 'sub-183', 'sub-184', 'sub-185']

    # subject_ids = ['sub-170', 'sub-173', 'sub-171']
    subject_ids = ['sub-01', 'sub-02', 'sub-03']
    # Parameters
    k = 3
    random_state = 42  # For reproducibility

    # Shuffle the subject list
    rng = np.random.default_rng(seed=random_state)
    shuffled_subjects = rng.permutation(subject_ids)

    # Split into k roughly equal folds
    folds = np.array_split(shuffled_subjects, k)

    # Optional: convert each fold to a list
    folds = [list(fold) for fold in folds]

 
    # exclude_subjects = ['sub-547', 'sub-639', 'sub-588', 'sub-171', 'sub-174', 'sub-184']
    # exclude_subjects = ['sub-547', 'sub-639', 'sub-588']
    exclude_subjects = ['sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10']
    # exclude_subjects = [ 'sub-174', 'sub-176', 'sub-179', 'sub-182', 'sub-177', 'sub-181', 'sub-183', 'sub-184', 'sub-185']
    
    chromo = "yuanyuan_v2"
    for fold in folds:
        subs = "_".join(fold)

        # Device configuration
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(device)

        train_csv_path, test_csv_path = create_train_test_segments(
            None,
            preprocessed_path, 
            test_subjects_list=fold, 
            exclude_subjects=exclude_subjects
        )
        train_csv = pd.read_csv(train_csv_path)
        test_csv = pd.read_csv(test_csv_path)

        train_dataset = fNIRSPreloadDataset(
            train_csv_path, chromo='HbO')
        test_dataset = fNIRSPreloadDataset(
            test_csv_path, mode="test", chromo='HbO')
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

        # Initialize model, loss, and optimizer
        model = CNN2DImage().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        
        train_losses = []
        train_accuracies = []
        test_losses = []
        test_accuracies = []
        test_f1_avgs = []
        test_f1s = []
        train_f1_avgs = []
        train_f1s = []

        # Training loop
        for epoch in range(num_epochs):

            train_loss = train_model(model, train_loader, criterion, optimizer, device)
            _, train_accuracy, train_f1, train_f1_avg, train_acc_avg = evaluate_model(model, train_loader, criterion, device)
            test_loss, test_accuracy, test_f1, test_f1_avg, test_acc_avg = evaluate_model(model, test_loader, criterion, device)


            # Store metrics
            train_losses.append(train_loss)
            train_accuracies.append(train_accuracy)
            test_losses.append(test_loss)
            test_accuracies.append(test_accuracy)
            test_f1_avgs.append(test_f1_avg)
            test_f1s.append(test_f1)
            train_f1_avgs.append(train_f1_avg)
            train_f1s.append(train_f1)

            print(f"Sub: {subs}, Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
                f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1:.4f}, "
                f"Test F1 Avg: {test_f1_avg:.4f}")
       
        res = {"train_loss": train_losses, "train_accuracy": train_accuracies,
                "test_loss": test_losses, "test_accuracy": test_accuracies, "test_f1": test_f1s,
                "test_f1_avg": test_f1_avgs, "test_acc_avg": test_acc_avg,
                "train_f1": train_f1s, "train_f1_avg": train_f1_avgs, "train_acc_avg": train_acc_avg}
        with open(f"/home/results/res_{subs}_{chromo}.pkl", "wb") as f:
            pickle.dump(res, f)

        # Save the trained model
        torch.save(model.state_dict(), f"/home/checkpoints/model_{subs}_{chromo}.pth")
        print("Model saved successfully!")


cuda
Excluding subjects: ['sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10']
Number of train subjects: 2
Preloading 1980 trials into memory...
Loaded 1980 trials into memory.
Preloading 111 trials into memory...
Loaded 111 trials into memory.
Sub: sub-03, Epoch [1/400], Train Loss: 0.7776, Train Accuracy: 0.5616, Test Loss: 0.6918, Test Accuracy: 0.5225, Test F1 Score: 0.5225, Test F1 Avg: 0.5238
Sub: sub-03, Epoch [2/400], Train Loss: 0.7815, Train Accuracy: 0.5561, Test Loss: 0.6902, Test Accuracy: 0.5045, Test F1 Score: 0.5045, Test F1 Avg: 0.5054
Sub: sub-03, Epoch [3/400], Train Loss: 0.7830, Train Accuracy: 0.5485, Test Loss: 0.6924, Test Accuracy: 0.4775, Test F1 Score: 0.4775, Test F1 Avg: 0.4780
Sub: sub-03, Epoch [4/400], Train Loss: 0.7752, Train Accuracy: 0.5414, Test Loss: 0.6840, Test Accuracy: 0.5045, Test F1 Score: 0.5045, Test F1 Avg: 0.5048
Sub: sub-03, Epoch [5/400], Train Loss: 0.7889, Train Accuracy: 0.5566, Test Loss: 0.7074, Test Accuracy: 0.48

KeyboardInterrupt: 